In [ ]:
!pip install langchain google-search-results langchain_core google-generativeai langchain_google_genai langchain_community

In [2]:
import os
from getpass import getpass

os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY") or getpass("enter langsmith key")

# below should not be changed
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
# you can change this as preferred
os.environ["LANGCHAIN_PROJECT"] = "deepdive-to-agents"

enter langsmith key··········


In [32]:
from langchain.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt_template = ChatPromptTemplate.from_messages([
    ("system",(
        "You're a helpful assistant. When answering a user's question "
        "you should first use one of the tools provided. After using a "
        "tool the tool output will be provided in the "
        "'scratchpad' below. If you have an answer in the "
        "scratchpad you should not use any more tools and "
        "instead answer directly to the user."
    )),
     MessagesPlaceholder(variable_name="chat_history"),
     ("human","{input}"),
     MessagesPlaceholder(variable_name="agent_scratchpad")

])

In [11]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    temperature=0.0,
    model="gemini-2.0-flash",
    api_key=getpass("Enter gemini api key: ")
)

Enter gemini api key: ··········


In [12]:
llm.invoke("Hello")

AIMessage(content='Hello! How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--b4ae988d-3180-4639-be38-082c9603692d-0', usage_metadata={'input_tokens': 1, 'output_tokens': 10, 'total_tokens': 11, 'input_token_details': {'cache_read': 0}})

In [14]:
from langchain_core.tools import tool

@tool
def add(x: float,y: float) -> float:
  """Add 'x' with 'y'."""
  return x+y
@tool
def subtract(x: float,y: float) -> float:
  """subtract 'y' from 'x'."""
  return x-y
@tool
def multiply(x: float,y: float) -> float:
  """multiply 'x' with 'y'."""
  return x*y
@tool
def exponentiate(x: float,y: float) -> float:
  """Raise 'x' to the power of 'y'."""
  return x**y


In [34]:
from langchain_core.runnables.base import RunnableSerializable

tools=[add,multiply,exponentiate,subtract]
agent : RunnableSerializable = (
    {
        "input": lambda x:x["input"],
        "chat_history": lambda x:x["chat_history"],
        "agent_scratchpad": lambda x: x.get("agent_scratchpad",[])
    }
    | prompt_template
    | llm.bind_tools(tools,tool_choice="any")
)

In [18]:
tool_call = agent.invoke({
    "input":"what is addition of 10 and 20"
    ,"chat_history":[]
})

In [19]:
tool_call

AIMessage(content='', additional_kwargs={'function_call': {'name': 'add', 'arguments': '{"y": 20.0, "x": 10.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--6605adb4-1a4f-48cf-b007-cffc4e454128-0', tool_calls=[{'name': 'add', 'args': {'y': 20.0, 'x': 10.0}, 'id': '966a60da-c94e-4157-8a74-e7ceed022ab9', 'type': 'tool_call'}], usage_metadata={'input_tokens': 147, 'output_tokens': 5, 'total_tokens': 152, 'input_token_details': {'cache_read': 0}})

In [20]:
name2tool = {tool.name:tool.func for tool in tools}
name2tool

{'add': <function __main__.add(x: float, y: float) -> float>,
 'multiply': <function __main__.multiply(x: float, y: float) -> float>,
 'exponentiate': <function __main__.exponentiate(x: float, y: float) -> float>,
 'subtract': <function __main__.subtract(x: float, y: float) -> float>}

In [21]:
tool_exec_content = name2tool[tool_call.tool_calls[0]["name"]](
    **tool_call.tool_calls[0]["args"]
)

In [22]:
tool_exec_content

30.0

###This is our answer and tool execution logic we feed this back into llm via scratchpad_placeholder along with tool_call_id

In [27]:
from langchain_core.messages import ToolMessage

tool_message = ToolMessage(
    content=f"The {tool_call.tool_calls[0]['name']} tool returned {tool_exec_content}",
    tool_call_id = tool_call.tool_calls[0]["id"]
)
out = agent.invoke({
    "input":"what is addition of 10 and 20",
    "chat_history":[],
    "agent_scratchpad":[tool_call,tool_message]
})

In [30]:
out

AIMessage(content='', additional_kwargs={'function_call': {'name': 'add', 'arguments': '{"y": 20.0, "x": 10.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--9a40fb73-dd73-4aa1-a484-8cc7f50cdf28-0', tool_calls=[{'name': 'add', 'args': {'y': 20.0, 'x': 10.0}, 'id': '1e112f58-0eb8-406c-bef8-2ab64c8647ca', 'type': 'tool_call'}], usage_metadata={'input_tokens': 163, 'output_tokens': 5, 'total_tokens': 168, 'input_token_details': {'cache_read': 0}})

#####Even after having the answer in the scratch pad the llm is calling tool , this is because we set tool_choice="any" there are two ways to solve this


1.   set tool_choice="auto" to tell the llm to choose tool or provide final answer
2.   create a final_answer tool



first lets try first one

In [37]:
agent: RunnableSerializable= (
    {
        "input":lambda x: x["input"],
        "chat_history":lambda x: x["chat_history"],
        "agent_scratchpad":lambda x: x.get("agent_scratchpad",[])
    }
    | prompt_template
    | llm.bind_tools(tools,tool_choice="auto")
)

Well repeat the process again , scratchpad is empty

In [38]:
tool_call = agent.invoke({"input":"what is the multiplication of 10 and 20","chat_history":[]})
tool_call

AIMessage(content='', additional_kwargs={'function_call': {'name': 'multiply', 'arguments': '{"y": 20.0, "x": 10.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--227932b7-e6eb-4acd-b098-cc646b30e426-0', tool_calls=[{'name': 'multiply', 'args': {'y': 20.0, 'x': 10.0}, 'id': '3bd5fb63-da48-45fa-820e-032ebd040b96', 'type': 'tool_call'}], usage_metadata={'input_tokens': 148, 'output_tokens': 5, 'total_tokens': 153, 'input_token_details': {'cache_read': 0}})

In [44]:
tool_output = name2tool[tool_call.tool_calls[0]["name"]](
  **tool_call.tool_calls[0]["args"]
)
tool_message = ToolMessage(
    content=f"The {tool_call.tool_calls[0]['name']} tool returned {tool_output}",
    tool_call_id=tool_call.tool_calls[0]["id"]
)
out=agent.invoke({
    "input":"what is the multiplication of 10 and 20",
     "chat_history":[],
    "agent_scratchpad":[tool_call,tool_message]
})
out

AIMessage(content='The multiplication of 10 and 20 is 200.0.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--23ee94ad-1d9b-4ca1-bc48-3dda6815fc10-0', usage_metadata={'input_tokens': 165, 'output_tokens': 19, 'total_tokens': 184, 'input_token_details': {'cache_read': 0}})

Now lets check with final_answer


In [52]:
@tool
def final_answer(answer: str, tools_used: list[str]) -> str:
    """Use this tool to provide a final answer to the user.
    The answer should be in natural language, as this will be provided to the user directly.
    The tools_used must include a list of tool names that were used within the 'scratchpad'.
    """
    return {
        "answer": answer,
        "tools_used": tools_used
    }

In [55]:
tools=[final_answer,add,multiply,exponentiate,subtract]
name2tool = {tool.name:tool.func for tool in tools}
agent: RunnableSerializable= (
    {
        "input":lambda x: x["input"],
        "chat_history":lambda x: x["chat_history"],
        "agent_scratchpad":lambda x: x.get("agent_scratchpad",[])
    }
    | prompt_template
    | llm.bind_tools(tools,tool_choice="any") #forcing llm to use the tool
)

In [56]:
tool_call = agent.invoke({"input":"what is the multiplication of 10 and 20","chat_history":[]})
tool_call

AIMessage(content='', additional_kwargs={'function_call': {'name': 'multiply', 'arguments': '{"y": 20.0, "x": 10.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--a00f8782-2660-45f0-b49f-e6012f7efaca-0', tool_calls=[{'name': 'multiply', 'args': {'y': 20.0, 'x': 10.0}, 'id': '4f80cb5e-3f38-41fd-8ebd-a022fa6d22c7', 'type': 'tool_call'}], usage_metadata={'input_tokens': 217, 'output_tokens': 5, 'total_tokens': 222, 'input_token_details': {'cache_read': 0}})

In [57]:
tool_output = name2tool[tool_call.tool_calls[0]["name"]](
  **tool_call.tool_calls[0]["args"]
)
tool_message = ToolMessage(
    content=f"The {tool_call.tool_calls[0]['name']} tool returned {tool_output}",
    tool_call_id=tool_call.tool_calls[0]["id"]
)
out=agent.invoke({
    "input":"what is the multiplication of 10 and 20",
     "chat_history":[],
    "agent_scratchpad":[tool_call,tool_message]
})
out.tool_calls["arguments"]

AIMessage(content='', additional_kwargs={'function_call': {'name': 'final_answer', 'arguments': '{"answer": "The multiplication of 10 and 20 is 200.0", "tools_used": ["multiply"]}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--41ae9092-2137-4bd2-ba4b-cadcb7d81802-0', tool_calls=[{'name': 'final_answer', 'args': {'answer': 'The multiplication of 10 and 20 is 200.0', 'tools_used': ['multiply']}, 'id': '31f16988-b095-4725-a697-2fc6449afa73', 'type': 'tool_call'}], usage_metadata={'input_tokens': 234, 'output_tokens': 25, 'total_tokens': 259, 'input_token_details': {'cache_read': 0}})

In [62]:
out.tool_calls[0]["args"]

{'answer': 'The multiplication of 10 and 20 is 200.0',
 'tools_used': ['multiply']}

#Building a custom Agent Execution loop

We've worked through each step of our agent code, but it doesn't run without us running every step. We must write a class to handle all the logic we just worked through.

In [71]:
from langchain_core.messages import BaseMessage,HumanMessage,AIMessage

class CustomAgentExecutor:
  chat_history: list[BaseMessage]

  def __init__(self,max_iteration:int =3):
    self.chat_history=[]
    self.max_iterations=max_iteration
    self.agent: RunnableSerializable= (
        {
            "input":lambda x: x["input"],
            "chat_history":lambda x: x["chat_history"],
            "agent_scratchpad":lambda x: x.get("agent_scratchpad",[])
        }
        | prompt_template
        | llm.bind_tools(tools,tool_choice="any") #forcing llm to use the tool again
    )
  def invoke(self,input:str) -> dict:
    #invoking the agent but we do this in a loop untill we ge the final answer
    count=0
    agent_scratchpad=[]
    while count<=self.max_iterations:
      tool_call = self.agent.invoke({
          "input":input,
          "chat_history":self.chat_history,
          "agent_scratchpad":agent_scratchpad
      })
      #add initial toolcall to agent scratchpad
      agent_scratchpad.append(tool_call)
      tool_name=tool_call.tool_calls[0]["name"] #assuming serial execution of tools
      tool_call_id=tool_call.tool_calls[0]["id"]
      tool_args = tool_call.tool_calls[0]["args"]
      tool_output = name2tool[tool_name](**tool_args)
      tool_message = ToolMessage(
          content=f"{tool_output}",
          tool_call_id=tool_call_id,
      )
      agent_scratchpad.append(tool_message)
      count+=1
      #add print to see intermediate steps
      print(f"{count} {tool_name}: {tool_args}")

      if tool_name=="final_answer":
        break
    final_answer=tool_output["answer"]
    self.chat_history.extend([
        HumanMessage(content=input),
        AIMessage(content=final_answer)
    ])
    return json.dumps(final_answer)

In [72]:
agent_executor = CustomAgentExecutor(max_iteration=5)
agent_executor.invoke(
"   A machine produces 15 units every hour. \
    After 3 hours, production is doubled by adding a second identical machine. \
    The machines then run together for another 4 hours. \
    Finally, the total production is squared to represent a performance score. \
    What is the final performance score?"
)

1 multiply: {'y': 3.0, 'x': 15.0}
2 multiply: {'y': 4.0, 'x': 15.0}
3 add: {'y': 60.0, 'x': 60.0}
4 add: {'y': 120.0, 'x': 45.0}
5 exponentiate: {'y': 2.0, 'x': 165.0}
6 final_answer: {'answer': 'The final performance score is 27225.', 'tools_used': ['multiply', 'add', 'exponentiate']}


'"The final performance score is 27225."'

In [73]:
agent_executor.chat_history

[HumanMessage(content='   A machine produces 15 units every hour.     After 3 hours, production is doubled by adding a second identical machine.     The machines then run together for another 4 hours.     Finally, the total production is squared to represent a performance score.     What is the final performance score?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='The final performance score is 27225.', additional_kwargs={}, response_metadata={})]